In [ ]:
import os
import re
import pdfplumber
import pandas as pd
import numpy as np
from pathlib import Path

def extract_data_from_pdf(pdf_path):
    """
    从PDF文件中提取表格数据并合并跨页的表格
    
    Args:
        pdf_path: PDF文件路径
        
    Returns:
        pandas.DataFrame: 合并后的完整表格数据
    """
    with pdfplumber.open(pdf_path) as pdf:
        # 提取第一页数据
        page1 = pdf.pages[0]
        text1 = page1.extract_text()
        
        # 分析第一页数据
        lines1 = text1.split('\n')
        header_index = -1
        
        # 先识别表头行的位置
        for i, line in enumerate(lines1):
            if '序号' in line and ('时间' in line or '日期' in line):
                header_index = i
                break
        
        if header_index == -1:
            raise ValueError("无法在PDF中找到表头行")
        
        # 提取表头
        header_line = lines1[header_index]
        # 规范化表头，确保有足够的分隔
        header_line = re.sub(r'(\d+)\s*([^\d\s])', r'\1 \2', header_line)  # 数字和非数字之间加空格
        headers1 = header_line.split()
        
        # 提取数据行和汇总行
        data1 = []
        summary_row = None
        
        for i in range(header_index + 1, len(lines1)):
            line = lines1[i]
            # 检测并单独处理汇总行
            if '汇总' in line:
                summary_parts = line.split()
                summary_row = summary_parts
                print(f"找到汇总行: {summary_row}")
            # 检测数据行（以数字开头的行）
            elif re.match(r'^\d+\s', line.strip()):
                # 规范化数据行，确保分隔合理
                line = re.sub(r'(\d+)\s+(\d{4}-\d{1,2}-\d{1,2})\s+(\d+)', r'\1 \2 \3', line)
                parts = line.split()
                
                # 确保日期和小时被正确分割
                if len(parts) > 2 and re.match(r'\d{4}-\d{1,2}-\d{1,2}', parts[1]):
                    # 日期和小时可能合并在一起，需要分开
                    data1.append(parts)
                    
        # 优化表头处理
        # 如果表头只有"序号"和"时间"的部分信息，补充完整
        if headers1 and len(headers1) >= 3:
            if not headers1[1] or headers1[1].isspace():
                headers1[1] = '日期'
            if not headers1[2] or headers1[2].isspace():
                headers1[2] = '小时'
        
        # 提取第二页数据
        headers2 = []
        data2 = []
        
        if len(pdf.pages) > 1:
            page2 = pdf.pages[1]
            text2 = page2.extract_text()
            
            # 分析第二页数据
            lines2 = text2.split('\n')
            
            # 第二页的第一行通常是表头
            if lines2:
                headers2 = lines2[0].split()
                
                # 提取第二页的数据行，从第二行开始
                for i in range(1, len(lines2)):
                    line = lines2[i]
                    # 只处理数据行，跳过可能的注释或页眉页脚
                    parts = line.split()
                    # 检查是否为有效的数据行（全部或大部分是数字）
                    valid_parts = [p for p in parts if re.match(r'^-?\d+(\.\d+)?$', p.strip())]
                    if len(valid_parts) >= len(parts) * 0.7:  # 如果70%以上是数字，认为是数据行
                        data2.append(parts)
        
        # 打印调试信息
        print(f"页面1 - 表头: {len(headers1)} 列，数据行数: {len(data1)}")
        if headers1:
            print(f"页面1表头: {headers1}")
        if data1:
            print(f"页面1首行: {data1[0]}")
        
        print(f"页面2 - 表头: {len(headers2)} 列，数据行数: {len(data2)}")
        if headers2:
            print(f"页面2表头: {headers2}")
        if data2:
            print(f"页面2首行: {data2[0]}")
        
        # 确定每页需要的列数
        required_cols1 = max([len(row) for row in data1] + [len(headers1)]) if data1 else (len(headers1) if headers1 else 0)
        required_cols2 = max([len(row) for row in data2] + [len(headers2)]) if data2 else (len(headers2) if headers2 else 0)
        
        print(f"第一页需要的列数: {required_cols1}, 第二页需要的列数: {required_cols2}")
        
        # 标准化表头和数据行的长度
        headers1 = standardize_length(headers1, required_cols1, prefix="第一页列")
        headers2 = standardize_length(headers2, required_cols2, prefix="第二页列")
        
        # 标准化数据行
        data1 = [standardize_length(row, required_cols1) for row in data1]
        data2 = [standardize_length(row, required_cols2) for row in data2]
        
        # 处理汇总行
        if summary_row:
            summary_row = standardize_length(summary_row, required_cols1)
        
        # 合并表头
        full_headers = headers1 + headers2
        
        # 确保两页的数据行数一致
        if data1 and data2:
            max_rows = max(len(data1), len(data2))
            data1 = ensure_row_count(data1, max_rows, required_cols1)
            data2 = ensure_row_count(data2, max_rows, required_cols2)
            
            # 合并数据行
            full_data = [data1[i] + data2[i] for i in range(max_rows)]
            
            # 如果有汇总行，添加到末尾
            if summary_row:
                # 为汇总行补充第二页的空数据
                full_summary = summary_row + [''] * required_cols2
                full_data.append(full_summary)
        elif data1:
            # 只有第一页有数据
            full_data = data1
            # 添加汇总行（如果有）
            if summary_row:
                full_data.append(summary_row)
            # 为每行添加第二页的空列
            full_data = [row + [''] * required_cols2 for row in full_data]
        elif data2:
            # 只有第二页有数据
            full_data = [[''] * required_cols1 + row for row in data2]
        else:
            # 两页都没有数据
            full_data = []
        
        # 创建DataFrame
        df = pd.DataFrame(full_data, columns=full_headers)
        
        # 转换数值列类型
        for col in df.columns:
            if col not in ['序号', '时间', '日期', '小时']:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        return df

def standardize_length(row, required_length, prefix="列"):
    """
    将行扩展到指定长度
    
    Args:
        row: 需要处理的行（列表）
        required_length: 需要的长度
        prefix: 当需要自动生成列名时使用的前缀
    
    Returns:
        列表: 标准化后的行
    """
    if not row:
        return [''] * required_length
    
    if len(row) < required_length:
        # 填充缺失的项
        return row + [''] * (required_length - len(row))
    
    # 如果长度超过要求，截断到要求长度
    return row[:required_length]

def ensure_row_count(data, required_rows, cols_per_row):
    """
    确保数据行数符合要求
    
    Args:
        data: 数据行的列表
        required_rows: 需要的行数
        cols_per_row: 每行的列数
    
    Returns:
        列表的列表: 标准化后的数据
    """
    if len(data) < required_rows:
        # 添加空行，使行数达到要求
        return data + [[''] * cols_per_row for _ in range(required_rows - len(data))]
    
    # 如果行数超过要求，截断到要求行数
    return data[:required_rows]

def process_all_pdfs(pdf_dir, output_path):
    """
    处理目录中的所有PDF文件，并将结果导出到一个Excel文件中
    
    Args:
        pdf_dir: 包含PDF文件的目录
        output_path: 输出的Excel文件路径
        
    Returns:
        bool: 是否成功处理了至少一个PDF文件
    """
    # 检查目录是否存在
    if not os.path.exists(pdf_dir):
        print(f"错误：目录 {pdf_dir} 不存在!")
        return False
    
    # 获取所有PDF文件
    pdf_files = [file for file in os.listdir(pdf_dir) if file.lower().endswith('.pdf')]
    
    if not pdf_files:
        print(f"警告：目录 {pdf_dir} 中没有找到PDF文件!")
        return False
    
    # 创建一个标志来跟踪是否至少有一个文件成功处理
    processed = False
    
    # 保存所有成功处理的数据
    processed_dfs = {}
    
    # 遍历所有PDF文件
    for file in pdf_files:
        pdf_path = os.path.join(pdf_dir, file)
        try:
            print(f"\n开始处理: {file}")
            # 提取PDF数据
            df = extract_data_from_pdf(pdf_path)
            
            # 获取不带扩展名的文件名作为sheet名称
            sheet_name = os.path.splitext(file)[0]
            if len(sheet_name) > 31:  # Excel限制sheet名称最长为31个字符
                sheet_name = sheet_name[:31]
            
            # 将数据添加到处理后的字典中
            processed_dfs[sheet_name] = df
            processed = True
            print(f"成功处理: {file}")
        except Exception as e:
            print(f"处理文件 {file} 时出错: {str(e)}")
            import traceback
            traceback.print_exc()
    
    # 如果至少有一个文件被处理，则保存到Excel
    if processed:
        # 确保输出目录存在
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            for sheet_name, df in processed_dfs.items():
                df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    return processed

# 在Jupyter notebook中运行时的代码
# 获取当前工作目录
current_dir = Path.cwd()
# pdf位于当前路径下的data文件夹中
data_dir = current_dir / 'data'
# 设置输出Excel文件路径
output_excel = current_dir / 'output' / '合并数据.xlsx'

# 处理所有PDF文件
if process_all_pdfs(data_dir, output_excel):
    print(f"处理完成，结果已保存到: {output_excel}")
else:
    # 如果没有成功处理任何PDF，我们创建一个简单的Excel文件
    df = pd.DataFrame({"消息": ["没有处理任何PDF文件"]})
    os.makedirs(os.path.dirname(output_excel), exist_ok=True)
    
    # 直接使用to_excel方法创建Excel文件，确保有一个可见的工作表
    df.to_excel(output_excel, sheet_name="信息", index=False)
    
    print("没有成功处理任何PDF文件，已创建一个带有消息的Excel文件。")
    print(f"请检查 {data_dir} 目录中是否有PDF文件。")

In [4]:
import pandas as pd
import os
from pathlib import Path
import re

def correct_headers_and_data(df):
    """
    修正表头和列数据:
    1. C列改为"小时"
    2. C～K列的表头平移至D～L列
    3. L列的内容被K列的替换掉
    
    Args:
        df: 待处理的DataFrame
        
    Returns:
        pandas.DataFrame: 修正后的DataFrame
    """
    # 获取原始列名列表
    columns = list(df.columns)
    
    # 确保至少有12列(即到L列)
    if len(columns) < 12:
        print(f"警告: 列数不足12列，当前只有{len(columns)}列，无法完全按要求调整")
        # 添加缺少的列
        for i in range(len(columns), 12):
            columns.append(f"Column_{i+1}")
            df[columns[-1]] = ""
    
    # 修正表头
    if len(columns) >= 12:
        # 保存C列数据以便后续恢复
        c_column_data = df.iloc[:, 2].copy()
        
        # 创建新表头
        new_columns = columns.copy()
        new_columns[2] = "小时"  # C列变为"小时"
        
        # 平移C～K列的表头至D～L列
        for i in range(2, 10+1):  # C到K列对应索引2到10
            new_columns[i+1] = columns[i]
        
        # L列的内容被K列替换
        new_columns[11] = columns[10]  # L列(索引11)使用K列(索引10)的列名
        
        # 修改DataFrame的列名
        df.columns = new_columns
        
        # 恢复C列(现在是"小时"列)的数据
        df.iloc[:, 2] = c_column_data
        
        print("表头已修正")
    else:
        print("列数不足，无法完全修正表头")
    
    return df

def remove_incomplete_rows(df):
    """
    删除表格中序号为24的数据行下面的内容不完整的行
    
    Args:
        df: 待处理的DataFrame
        
    Returns:
        pandas.DataFrame: 处理后的DataFrame
    """
    # 查找序号为24的行
    row_24_index = None
    
    # 检查第一列是否为"序号"列
    if "序号" in df.columns:
        # 使用列名查找
        try:
            row_24_indices = df[df["序号"] == 24].index
            if not row_24_indices.empty:
                row_24_index = row_24_indices[0]
        except:
            pass
    
    # 如果通过列名没找到，尝试直接检查第一列值
    if row_24_index is None:
        for idx, value in enumerate(df.iloc[:, 0]):
            try:
                if int(value) == 24:
                    row_24_index = idx
                    break
            except:
                pass
    
    # 删除24行之后的所有行(不包括24行本身)
    if row_24_index is not None:
        # 保留到第24行(包括)
        df_filtered = df.iloc[:row_24_index+1]
        print(f"已删除序号24之后的行，从{len(df)}行减少到{len(df_filtered)}行")
        return df_filtered
    else:
        print("未找到序号为24的行，保持原数据不变")
        return df

def add_summary_row(df):
    """
    将各列数据汇总(求和)，添加到最后一行
    
    Args:
        df: 待处理的DataFrame
        
    Returns:
        pandas.DataFrame: 添加了汇总行的DataFrame
    """
    # 创建一个新行用于存放汇总结果
    summary_row = {}
    
    # 第一列(序号列)设为"汇总"
    first_col = df.columns[0]
    summary_row[first_col] = "汇总"
    
    # 时间列留空
    if len(df.columns) > 1:
        summary_row[df.columns[1]] = ""
    
    # 小时列留空
    if len(df.columns) > 2:
        summary_row[df.columns[2]] = ""
    
    # 对其他列进行求和
    for col in df.columns[3:]:  # 跳过序号、时间和小时列
        try:
            # 尝试将列数据转换为数值并求和
            numeric_data = pd.to_numeric(df[col], errors='coerce')
            col_sum = numeric_data.sum(skipna=True)
            if pd.isna(col_sum):
                summary_row[col] = ""
            else:
                # 如果是整数，去掉小数部分
                if col_sum == int(col_sum):
                    summary_row[col] = int(col_sum)
                else:
                    summary_row[col] = col_sum
        except:
            # 如果无法计算总和，设为空
            summary_row[col] = ""
    
    # 将汇总行添加到DataFrame
    df_with_summary = pd.concat([df, pd.DataFrame([summary_row])], ignore_index=True)
    
    print("已添加汇总行")
    return df_with_summary

def process_excel_file(input_path, output_path):
    """
    处理Excel文件:
    1. 修正表头
    2. 删除多余行
    3. 添加汇总行
    
    Args:
        input_path: 输入Excel文件路径
        output_path: 输出Excel文件路径
    """
    print(f"开始处理Excel文件: {input_path}")
    
    # 确保输出目录存在
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    try:
        # 读取所有sheet
        excel_file = pd.ExcelFile(input_path)
        sheet_names = excel_file.sheet_names
        
        if not sheet_names:
            print("Excel文件不包含任何Sheet")
            return
        
        print(f"发现{len(sheet_names)}个Sheet: {', '.join(sheet_names)}")
        
        # 使用ExcelWriter保存处理后的数据
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            for sheet_name in sheet_names:
                try:
                    print(f"\n处理Sheet: {sheet_name}")
                    # 读取当前sheet
                    df = pd.read_excel(input_path, sheet_name=sheet_name)
                    
                    # 修正表头和列数据
                    df = correct_headers_and_data(df)
                    
                    # 删除不完整行
                    df = remove_incomplete_rows(df)
                    
                    # 添加汇总行
                    df = add_summary_row(df)
                    
                    # 保存到新Excel
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
                    
                    print(f"完成处理Sheet: {sheet_name}")
                except Exception as e:
                    print(f"处理Sheet {sheet_name} 时出错: {str(e)}")
                    import traceback
                    traceback.print_exc()
        
        print(f"\n所有数据已处理完成并保存到: {output_path}")
        
    except Exception as e:
        print(f"处理Excel文件时出错: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    # 设置输入和输出文件路径
    current_dir = Path.cwd()
    input_excel = current_dir / "output" / "合并数据.xlsx"
    output_excel = current_dir / "output" / "修正后数据.xlsx"
    
    # 处理Excel文件
    process_excel_file(input_excel, output_excel)

开始处理Excel文件: /Users/henri/Documents/VS-Code-Git/Open-Learning/Projects/PDF-data-get/output/合并数据.xlsx
发现50个Sheet: 20250204, 20250210, 20250211, 20250205, 20250213, 20250207, 20250206, 20250212, 20250216, 20250202, 20250203, 20250217, 20250201, 20250215, 20250228, 20250214, 20250313, 20250307, 20250306, 20250312, 20250304, 20250310, 20250311, 20250305, 20250301, 20250315, 20250314, 20250316, 20250302, 20250303, 20250317, 20250319, 20250318, 20250320, 20250308, 20250309, 20250321, 20250323, 20250322, 20250219, 20250224, 20250218, 20250226, 20250227, 20250223, 20250222, 20250220, 20250208, 20250209, 20250221

处理Sheet: 20250204
表头已修正
已删除序号24之后的行，从25行减少到24行
已添加汇总行
完成处理Sheet: 20250204

处理Sheet: 20250210
表头已修正
已删除序号24之后的行，从25行减少到24行
已添加汇总行
完成处理Sheet: 20250210

处理Sheet: 20250211
表头已修正
已删除序号24之后的行，从25行减少到24行
已添加汇总行
完成处理Sheet: 20250211

处理Sheet: 20250205
表头已修正
已删除序号24之后的行，从25行减少到24行
已添加汇总行
完成处理Sheet: 20250205

处理Sheet: 20250213
表头已修正
已删除序号24之后的行，从25行减少到24行
已添加汇总行
完成处理Sheet: 20250213

处理Sheet: 202